#### Yu Han, June 2023.
We used logistic regression to classify neuron cells with deletion vs. w/o deletion. Feature importance score is printed. 

mann-whitney-u-test was adopted to run feature by feature analysis between the deletion group and the control group to find significant features.

Combining morph features and branching features.

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import math
import statistics
import random
import scipy.stats as ss

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit

import warnings
warnings.filterwarnings("ignore")
from pycytominer import aggregate
import statsmodels.stats.multitest

### logistic regression

In [2]:
#read morph feature profile data of two plates 
df_p72=pd.read_csv('../2019_05_28_Neuronal_Cell_Painting/profiles/2022_03_03_NCP_NEURONS_2_20x/BR00132672/BR00132672_normalized_feature_select_batch.csv.gz')
df_p73=pd.read_csv('../2019_05_28_Neuronal_Cell_Painting/profiles/2022_03_03_NCP_NEURONS_2_20x/BR00132673/BR00132673_normalized_feature_select_batch.csv.gz')
df_neuron=pd.concat([df_p72,df_p73])
df_neuron=df_neuron.drop(['Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel'], axis=1)
df_neuron.head(2)

,Metadata_plate_map_name,Metadata_EXPERIMENT_NAME,Metadata_DENSITY,Metadata_LINE_ID,Metadata_GENOTYPE,Metadata_Plate,Metadata_Well,Metadata_Site_Count,Metadata_Object_Count,Cells_AreaShape_Compactness,...,Nuclei_Texture_InverseDifferenceMoment_AGP_20_00,Nuclei_Texture_InverseDifferenceMoment_AGP_5_02,Nuclei_Texture_InverseDifferenceMoment_Brightfield_20_01,Nuclei_Texture_InverseDifferenceMoment_DNA_20_03,Nuclei_Texture_InverseDifferenceMoment_Mito_20_02,Nuclei_Texture_InverseDifferenceMoment_RNA_20_01,Nuclei_Texture_InverseDifferenceMoment_RNA_5_01,Nuclei_Texture_SumAverage_AGP_20_03,Nuclei_Texture_SumAverage_ER_20_02,Nuclei_Texture_SumVariance_Brightfield_20_00
0,NCP_2022_03_03_METADATA,2022_03_03_NCP_NEURONS_2,3500,1,DELETION,BR00132672,A01,3,144,-0.17229,...,-0.065279,1.0098,-1.32230,0.083103,0.41479,0.29334,1.2459,-0.54579,-0.98414,2.7363
1,NCP_2022_03_03_METADATA,2022_03_03_NCP_NEURONS_2,3500,1,DELETION,BR00132672,A02,3,83,-0.33451,...,0.596690,1.5056,-0.60581,0.862090,0.25006,1.02780,2.3084,-0.82466,-1.75760,-0.3397


In [3]:
#read branch metric profile
df_branch=pd.read_csv('profile/neuron_branch_normalized.csv')

In [4]:
#merge morph and branch profile
df_merge=pd.merge(df_neuron, df_branch, on =["Metadata_Plate",'Metadata_Well'])
#remove 23, 24, 25, 26 from Metadata_LINE_ID
#remove isogenic cell lines
df_merge=df_merge[~df_merge.Metadata_LINE_ID_x.isin([23,24,25,26])]
df_merge.head(2)

,Metadata_plate_map_name_x,Metadata_EXPERIMENT_NAME_x,Metadata_DENSITY_x,Metadata_LINE_ID_x,Metadata_GENOTYPE_x,Metadata_Plate,Metadata_Well,Metadata_Site_Count_x,Metadata_Object_Count_x,Cells_AreaShape_Compactness,...,Metadata_EXPERIMENT_NAME_y,Metadata_DENSITY_y,Metadata_LINE_ID_y,Metadata_GENOTYPE_y,Metadata_Site_Count_y,Metadata_Object_Count_y,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,NCP_2022_03_03_METADATA,2022_03_03_NCP_NEURONS_2,3500,1,DELETION,BR00132672,A01,3,144,-0.17229,...,2022_03_03_NCP_NEURONS_2,3500,1,DELETION,3,144,-0.95647,-0.88478,-0.73272,-0.81801
1,NCP_2022_03_03_METADATA,2022_03_03_NCP_NEURONS_2,3500,1,DELETION,BR00132672,A02,3,83,-0.33451,...,2022_03_03_NCP_NEURONS_2,3500,1,DELETION,3,83,-0.64866,-0.46213,-0.02024,-0.61935


In [5]:
#save df_merge as csv
df_merge.to_csv('profile/neuron_morph_branch_merge.csv', index=False)

In [6]:
#code deletion as 1 and control as 0. 
df_merge.Metadata_GENOTYPE_x[df_merge.Metadata_GENOTYPE_x == 'DELETION'] = 1
df_merge.Metadata_GENOTYPE_x[df_merge.Metadata_GENOTYPE_x == 'CONTROL'] = 0

In [7]:
#select all feature columns
df_merge_feature=df_merge.loc[:,~df_merge.columns.str.startswith('Metadata')]

In [8]:
### logistic regression 
#define X and y
X=df_merge_feature.values
y=df_merge['Metadata_GENOTYPE_x'].tolist()
y0=np.array(y)

#split training and testing 
#using GroupShuffleSplit to ensure the same patient won't appear in train and test set at the same time
gss = GroupShuffleSplit(n_splits=2, train_size=.8, random_state=42)
gss.get_n_splits()
for train_idx, test_idx in gss.split(X, y0, df_merge['Metadata_LINE_ID_x']):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y0[train_idx], y0[test_idx]

#fit the model
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)

#print acc score
score = logisticRegr.score(X_test, y_test)
score

0.7777777777777778

In [9]:
#print feature importance values
neuron_feature_weights = pd.DataFrame(logisticRegr.coef_[0], columns=['weight'], index=df_merge_feature.columns)
neuron_feature_weights.reindex(neuron_feature_weights['weight'].abs().sort_values(ascending=False).index).head(10)

,weight
Nuclei_Correlation_Correlation_RNA_AGP,0.147156
Nuclei_Correlation_Correlation_ER_AGP,0.124252
Nuclei_RadialDistribution_FracAtD_Brightfield_3of4,0.122482
Nuclei_AreaShape_Zernike_6_4,-0.121856
Cytoplasm_Intensity_MassDisplacement_Brightfield,0.119314
Nuclei_Texture_Contrast_ER_20_02,-0.115858
Cytoplasm_Correlation_K_DNA_AGP,-0.108255
Nuclei_Texture_AngularSecondMoment_Brightfield_20_01,-0.104403
Cytoplasm_Texture_Contrast_Brightfield_10_02,-0.099547
Cytoplasm_Granularity_2_Mito,0.099218


### mann-whitney-u-test

In [10]:
#mean profile based on patient ID and genotype
df_merge_mean= aggregate(
                    population_df=df_merge, 
                    strata=['Metadata_LINE_ID_x', 'Metadata_GENOTYPE_x'], 
                    features='infer', 
                    operation='mean' 
                    )
df_merge_mean.head(2)

,Metadata_LINE_ID_x,Metadata_GENOTYPE_x,Cells_AreaShape_Compactness,Cells_AreaShape_MedianRadius,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_3_3,Cells_AreaShape_Zernike_4_0,...,Nuclei_Texture_InverseDifferenceMoment_Mito_20_02,Nuclei_Texture_InverseDifferenceMoment_RNA_20_01,Nuclei_Texture_InverseDifferenceMoment_RNA_5_01,Nuclei_Texture_SumAverage_AGP_20_03,Nuclei_Texture_SumAverage_ER_20_02,Nuclei_Texture_SumVariance_Brightfield_20_00,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,1,1,0.051249,-0.623481,-0.012366,0.131600,0.390005,-0.640063,0.378335,0.733289,...,0.533359,-0.263681,0.341041,-0.751126,-0.444946,-0.485764,-0.576519,-0.490640,-0.049856,-0.717221
1,2,1,-0.232959,0.745598,-0.024121,0.244952,0.223665,-0.008189,0.085162,0.351442,...,-0.391601,0.754717,1.302866,0.003830,-0.587460,0.822778,0.244571,0.299023,0.392656,0.279753


In [11]:
#select all feature columns
df_merge_feature_u=df_merge_mean.loc[:,~df_merge_mean.columns.str.startswith('Metadata')]

In [12]:
#create feature list
feature_list=list(df_merge_feature_u.columns)

In [13]:
#create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0],columns=feature_list)

In [14]:
#run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(df_merge_mean.query('Metadata_GENOTYPE_x == 1')[feat],          
           df_merge_mean.query('Metadata_GENOTYPE_x == 0')[feat])
    list_p.append(p)
    list_u.append(u)

In [15]:
list_p_fdr=statsmodels.stats.multitest.fdrcorrection(list_p, alpha=0.05, method='indep', is_sorted=False)

In [16]:
#store p value to each feature
df_p_values_feature.iloc[0] =list_p_fdr[1]
df_p_values_feature

,Cells_AreaShape_Compactness,Cells_AreaShape_MedianRadius,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_3_3,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_4_2,Cells_AreaShape_Zernike_4_4,...,Nuclei_Texture_InverseDifferenceMoment_Mito_20_02,Nuclei_Texture_InverseDifferenceMoment_RNA_20_01,Nuclei_Texture_InverseDifferenceMoment_RNA_5_01,Nuclei_Texture_SumAverage_AGP_20_03,Nuclei_Texture_SumAverage_ER_20_02,Nuclei_Texture_SumVariance_Brightfield_20_00,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel
0,0.460177,0.460177,0.478117,0.49273,0.428887,0.454858,0.428887,0.470448,0.426227,0.460177,...,0.266918,0.470448,0.251104,0.470448,0.470448,0.464511,0.464511,0.460177,0.476702,0.489921


In [17]:
#select features with p value smaller than .001
df_p_values_feature= df_p_values_feature.drop(df_p_values_feature.
                         columns[df_p_values_feature.iloc[-1,:]>0.05],axis=1)
df_p_values_feature.columns

Index([], dtype='object')